In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import segmentation_models as sm

from IPython.display import clear_output

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os
import shutil
import cv2
import time
import glob

from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from scipy import io
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, exposure
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import cv2
import glob
import os

In [ ]:
# read in and prepare augmented images for train

augmented_images =[]
augmented_labels = []
IMG_SIZE = 299
for class_id in [1,2,3,4]:
    suffix = 'c' + str(class_id)+ '/'
    directory_path = glob.glob('data/segmentation/train_aug/' + suffix)[0]
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
        image_id = img_path.split('/')[-1]
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        #resize images
        image_resized=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
        #print(np.array(image_resized).shape)
       
        #The input data have to be converted from 3 dimensional format to 1 dimensional format
#        image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)
        # Data Normalization
        # Conversion to float
#        image_flat=image_flat.astype('float32')
        # Normalization (In the RGB color space the red, green and blue have integer values from 0 to 255)
        image_resized = image_resized/255.0
        augmented_images.append(image_resized)
        augmented_labels.append(class_id)
  #      print(np.array(augmented_images).shape)
   #     break

In [ ]:
x_train = augmented_images
y_train = augmented_labels

In [ ]:
y_train = pd.Series(y_train)

In [ ]:
x_train = np.array(x_train)

In [ ]:
x_train.shape

In [ ]:
import sys
sys.path.insert(0, 'Python_Scripts')

import util
df = pd.read_csv('data/train_complete.csv')
util.isolate_single_defects(df)
df2 = df.query('ClassId != 0')

In [ ]:
initial_images =[]

IMG_SIZE = 128
for image_id in sorted(df2['ImageId']):
    image = io.imread('data/single_defect_train_images/' + image_id)
    #resize images
    image_resized=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
    #The input data have to be converted from 3 dimensional format to 1 dimensional format
    image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)
    # Data Normalization
    # Conversion to float
    image_flat=image_flat.astype('float32')
    # Normalization (In the RGB color space the red, green and blue have integer values from 0 to 255)
    image_flat = image_flat/255.0
    initial_images.append([image_id,image_flat[0]])

In [ ]:
initial_images = pd.DataFrame(initial_images , columns = ['ImageId', 'flattened_images'])
split_initial_images = pd.DataFrame(initial_images['flattened_images'].tolist())
initial_images_complete = pd.concat([initial_images, split_initial_images], axis=1)
initial_images_complete.drop(['flattened_images'], axis=1, inplace=True)
initial_images_complete = pd.merge(initial_images_complete, df[['ImageId','ClassId']], on='ImageId')
y = initial_images_complete['ClassId']

In [ ]:
#y = pd.Series(y)
y = pd.Series(y)

In [ ]:
images =[]

IMG_SIZE = 299
for image_id in sorted(df2['ImageId']):
    image = cv2.imread('data/single_defect_train_images/' + image_id,cv2.IMREAD_COLOR)
    #resize images
    image_resized=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
    #The input data have to be converted from 3 dimensional format to 1 dimensional format
  #  image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)
    # Data Normalization
    # Conversion to float
 #   image_flat=image_flat.astype('float32')
    # Normalization (In the RGB color space the red, green and blue have integer values from 0 to 255)
    image_resized = image_resized/255.0
  #  images.append([image_id,image_resized[0]])
    images.append(image_resized)

In [ ]:
x_val = images[1:2000]


In [ ]:
y_val = y[1:2000]

In [ ]:
y_val.shape

In [ ]:
x_val = np.array(x_val)

In [ ]:
model = keras.applications.InceptionResNetV2(weights='imagenet', input_shape=(299,299,3), include_top=False)
#model.load_weights('/kaggle/input/inceptionresnetv2/inception_resent_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.trainable=False

x=model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(128,activation='relu')(x)
x=tf.keras.layers.Dense(64,activation='relu')(x) 
outputs=tf.keras.layers.Dense(4,activation='sigmoid')(x) #final layer multilabel classifier

model_multi=tf.keras.Model(inputs=model.input,outputs=outputs) 

In [ ]:
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model_multi.compile(optimizer=OPTIMIZER, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy']) 

In [ ]:
preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input

In [ ]:

x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
y_val

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
#reshape the 1-D array to 2-D as fit_transform expects 2-D and finally fit the object 
y_train = onehotencoder.fit_transform(y_train.reshape(-1,1)).toarray()
y_val = onehotencoder.fit_transform(y_val.reshape(-1,1)).toarray()

In [ ]:
y_val.shape

In [ ]:
y_val

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
history=model_multi.fit(x_train, 
                  y_train,
                  batch_size=16, 
                  epochs=17,
                  verbose=1,
                  validation_data=(x_val, y_val)
                 )

In [ ]:
y_pred = model_multi.predict(x_val)

In [ ]:
y_pred[1:20]